In [1]:
import requests
import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
import pandas as pd
import matplotlib.pyplot as plt
!pip install pandas_market_calendars
import requests
from datetime import datetime as dt
import pandas_market_calendars as mcal
from datetime import datetime as dt

In [2]:
# Create a calendar
nyse = mcal.get_calendar('NYSE')

In [28]:
timespan = nyse.schedule(start_date='2020-09-15', end_date='2021-03-15')

In [29]:
timespan.head()

,market_open,market_close
2020-09-15,2020-09-15 13:30:00+00:00,2020-09-15 20:00:00+00:00
2020-09-16,2020-09-16 13:30:00+00:00,2020-09-16 20:00:00+00:00
2020-09-17,2020-09-17 13:30:00+00:00,2020-09-17 20:00:00+00:00
2020-09-18,2020-09-18 13:30:00+00:00,2020-09-18 20:00:00+00:00
2020-09-21,2020-09-21 13:30:00+00:00,2020-09-21 20:00:00+00:00


In [30]:
times = list(timespan.index.values)

In [31]:
dates = []
for date in range(len(times)):
    ts = pd.to_datetime(str(times[date]))
    d = ts.strftime('%Y-%m-%d')
    dates.append(d)

In [32]:
dates

['2020-09-15',
 '2020-09-16',
 '2020-09-17',
 '2020-09-18',
 '2020-09-21',
 '2020-09-22',
 '2020-09-23',
 '2020-09-24',
 '2020-09-25',
 '2020-09-28',
 '2020-09-29',
 '2020-09-30',
 '2020-10-01',
 '2020-10-02',
 '2020-10-05',
 '2020-10-06',
 '2020-10-07',
 '2020-10-08',
 '2020-10-09',
 '2020-10-12',
 '2020-10-13',
 '2020-10-14',
 '2020-10-15',
 '2020-10-16',
 '2020-10-19',
 '2020-10-20',
 '2020-10-21',
 '2020-10-22',
 '2020-10-23',
 '2020-10-26',
 '2020-10-27',
 '2020-10-28',
 '2020-10-29',
 '2020-10-30',
 '2020-11-02',
 '2020-11-03',
 '2020-11-04',
 '2020-11-05',
 '2020-11-06',
 '2020-11-09',
 '2020-11-10',
 '2020-11-11',
 '2020-11-12',
 '2020-11-13',
 '2020-11-16',
 '2020-11-17',
 '2020-11-18',
 '2020-11-19',
 '2020-11-20',
 '2020-11-23',
 '2020-11-24',
 '2020-11-25',
 '2020-11-27',
 '2020-11-30',
 '2020-12-01',
 '2020-12-02',
 '2020-12-03',
 '2020-12-04',
 '2020-12-07',
 '2020-12-08',
 '2020-12-09',
 '2020-12-10',
 '2020-12-11',
 '2020-12-14',
 '2020-12-15',
 '2020-12-16',
 '2020-12-

In [39]:
ticker = 'RIOT'
range_agg = '/range/1/minute/'
unadjusted = 'unadjusted=true'
sort = '&sort=asc'
limit = '&limit=50000'
POLYGON_URL = 'https://api.polygon.io/v2/aggs/ticker/'

In [40]:
import data_config
datetime = []
open_price = []
high_price = []
low_price = []
close_price = []
volume = []
for date in dates:
    polygon = POLYGON_URL + ticker + range_agg + date + '/' + date + '?' + unadjusted + sort + limit + data_config.POLYGON_API_KEY
    r_polygon = requests.get(polygon)
    polygon_response = r_polygon.json()
    for aggs in range(len(polygon_response['results'])):
        datetime.append(polygon_response['results'][aggs]['t'])
        open_price.append(polygon_response['results'][aggs]['o'])
        high_price.append(polygon_response['results'][aggs]['h'])
        low_price.append(polygon_response['results'][aggs]['l'])
        close_price.append(polygon_response['results'][aggs]['c'])
        volume.append(polygon_response['results'][aggs]['v'])

In [41]:
df_ticks = pd.DataFrame(list(zip(datetime,open_price,high_price,low_price, close_price,volume)),
columns =['datetime', 'open', 'high', 'low', 'close', 'volume'])

In [42]:
df_ticks.head()

,datetime,open,high,low,close,volume
0,1600167420000,3.07,3.07,3.07,3.07,150.0
1,1600167480000,3.07,3.07,3.07,3.07,350.0
2,1600169940000,3.09,3.09,3.07,3.07,4000.0
3,1600170360000,3.11,3.11,3.11,3.11,400.0
4,1600170660000,3.11,3.11,3.11,3.11,100.0


In [43]:
len(df_ticks)

86435

In [44]:
df_ticks.to_csv('stock_prices__min_test_RIOT.csv',sep = ",", index=True)

In [45]:
len(dates)

125